# Extract snow depth along a trajectory __UNDER CONSTRUCTION__

## Load sea ice trajectory
I use an icetrack extracted using the [NSIDC ice tracking tool](http://icemotion.labs.nsidc.org/SITU/#).  The tool outputs a csv file.  This is easily read using `pandas`.  I want a dataframe indexed by date.  I use `parse_dates={'time': ['year','month','day'}` and `index_col='time'` to create a date index.  I only need latitude and longitude so I use `usecols` to extract latitude and longitude.  However, because `year`, `month` and `day` are used to generate the date index, these columns must be included in usecols.

In [1]:
import os
import sys
sys.path.append("../sunlight")  # Add `sunlight` to $PYTHONPATH

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from readers.snowmodel import read as read_snowmodel

import pandas as pd

In [ ]:
trackfile = '../Data/icetracks.20040403.csv'
df = pd.read_csv(trackfile, header=0, parse_dates={'time': ['year', 'month', 'day']}, index_col='time', usecols=['lat', 'lon', 'year', 'month', 'day'])
df.head()

The snow depth data are on a grid in the NSIDC EASE-Grid projection.  So the latitude and longitude coordinates need to be transformed to projected `(x,y)` coordinates.  `cartopy.crs.Projection` classes have a `transform_points` method that can do this easily.  

Not a link to the EASE-Grid but includes info about the grid https://nsidc.org/data/nsidc-0116

Information about EASE-Grid is available [here](https://nsidc.org/data/ease)

In [ ]:
SphericalEarth = ccrs.Globe(semimajor_axis=6371228., semiminor_axis=6371228.)
NSIDC_EASE = ccrs.LambertAzimuthalEqualArea(central_latitude=90., central_longitude=0., globe=SphericalEarth)

xyz = NSIDC_EASE.transform_points(ccrs.PlateCarree(), df.lon.values, df.lat.values)
x = xyz[:,0]
y = xyz[:,1]

## Load snow depth

In [ ]:
DATAPATH = os.path.join(os.path.sep, "home", "apbarret", "Data", "Snow_on_seaice", "SnowModelOutput")
datafile = os.path.join(DATAPATH, "MERRA2", "snod.gdat")
ctlfile = os.path.join(DATAPATH, "MERRA2", "SM_snod_merra2_01Aug1980-31Jul2018.ctl")
#datafile = os.path.join(DATAPATH, "ERA5", snod.gdat")
#ctlfile = os.path.join(DATAPATH, "ERA5", SM_snod_era5_01Aug1980-31Jul2018.ctl")

snowDepth = read_snowmodel(datafile, ctlfile)
snowDepth

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection=NSIDC_EASE)
ax.set_extent([-4524683.8, 4524683.8, -4524683.8, 4524683.8], NSIDC_EASE)
ax.coastlines()
ax.gridlines()
snowDepth.sel(time="2010-0")
ax.plot(x, y, transform=NSIDC_NorthPolarStereo)
ax.plot(df.lon.values, df.lat.values, transform=ccrs.PlateCarree())

The data are weekly and need to be interpolated to daily to extract the data.  __TBD__